In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
df = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")
num_col = list(df.describe().columns)
col_categorical = list(set(df.columns).difference(num_col))
remove_list = ['EmployeeCount', 'EmployeeNumber', 'StandardHours']
col_numerical = [e for e in num_col if e not in remove_list]
attrition_to_num = {'Yes': 0,
                    'No': 1}
df['Attrition_num'] = df['Attrition'].map(attrition_to_num)
col_categorical.remove('Attrition')
df_cat = pd.get_dummies(df[col_categorical])
X = pd.concat([df[col_numerical], df_cat], axis=1)
y = df['Attrition_num']

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [12]:
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, roc_auc_score
def print_score(clf, X_train, X_test, y_train, y_test, train=True):
    '''
    v0.1 Follow the scikit learn library format in terms of input
    print the accuracy score, classification report and confusion matrix of classifier
    '''
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_train)
    if train:
        '''
        training performance
        '''
        print("Train Result:\n")
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_train, clf.predict(X_train))))
        print("Classification Report: \n {}\n".format(classification_report(y_train, clf.predict(X_train))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_train, clf.predict(X_train))))
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_train), 
                                                        lb.transform(clf.predict(X_train)))))

        #cv_res = cross_val_score(clf, X_train, y_train, cv=10, scoring='accuracy')
        #print("Average Accuracy: \t {0:.4f}".format(np.mean(cv_res)))
        #print("Accuracy SD: \t\t {0:.4f}".format(np.std(cv_res)))
        
    elif train==False:
        '''
        test performance
        '''
        res_test = clf.predict(X_test)
        print("Test Result:\n")        
        print("accuracy score: {0:.4f}\n".format(accuracy_score(y_test, clf.predict(X_test))))
        print("Classification Report: \n {}\n".format(classification_report(y_test, clf.predict(X_test))))
        print("Confusion Matrix: \n {}\n".format(confusion_matrix(y_test, clf.predict(X_test))))      
        print("ROC AUC: {0:.4f}\n".format(roc_auc_score(lb.transform(y_test), lb.transform(res_test))))
        

# Model 1:Decision Tree

In [14]:
from sklearn.tree import DecisionTreeClassifier
model_1=DecisionTreeClassifier()
model_1.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [15]:
print_score(model_1,X_train,X_test,y_train,y_test,train=True)
print_score(model_1,X_train,X_test,y_train,y_test,train=False)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       180
           1       1.00      1.00      1.00       922

    accuracy                           1.00      1102
   macro avg       1.00      1.00      1.00      1102
weighted avg       1.00      1.00      1.00      1102


Confusion Matrix: 
 [[180   0]
 [  0 922]]

ROC AUC: 1.0000

Test Result:

accuracy score: 0.8207

Classification Report: 
               precision    recall  f1-score   support

           0       0.43      0.47      0.45        57
           1       0.90      0.88      0.89       311

    accuracy                           0.82       368
   macro avg       0.67      0.68      0.67       368
weighted avg       0.83      0.82      0.82       368


Confusion Matrix: 
 [[ 27  30]
 [ 36 275]]

ROC AUC: 0.6790



# Model 2:Extratrees

In [17]:
from sklearn.tree import ExtraTreeClassifier

In [18]:
model_2=ExtraTreeClassifier()

In [19]:
model_2.fit(X_train,y_train)

ExtraTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    min_impurity_decrease=0.0, min_impurity_split=None,
                    min_samples_leaf=1, min_samples_split=2,
                    min_weight_fraction_leaf=0.0, random_state=None,
                    splitter='random')

In [20]:
print_score(model_2,X_train,X_test,y_train,y_test,train=True)
print_score(model_2,X_train,X_test,y_train,y_test,train=False)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       180
           1       1.00      1.00      1.00       922

    accuracy                           1.00      1102
   macro avg       1.00      1.00      1.00      1102
weighted avg       1.00      1.00      1.00      1102


Confusion Matrix: 
 [[180   0]
 [  0 922]]

ROC AUC: 1.0000

Test Result:

accuracy score: 0.7962

Classification Report: 
               precision    recall  f1-score   support

           0       0.29      0.21      0.24        57
           1       0.86      0.90      0.88       311

    accuracy                           0.80       368
   macro avg       0.57      0.56      0.56       368
weighted avg       0.77      0.80      0.78       368


Confusion Matrix: 
 [[ 12  45]
 [ 30 281]]

ROC AUC: 0.5570



In [21]:
en=pd.DataFrame()

In [22]:
en['model_1']=pd.DataFrame(model_1.predict_proba(X_train))[1]

In [23]:
en['model_2']=pd.DataFrame(model_2.predict_proba(X_train))[1]

In [24]:
col_name=en.columns

In [25]:
en=pd.concat([en,pd.DataFrame(y_train).reset_index(drop=True)],axis=1)

In [26]:
en.head()

,model_1,model_2,Attrition_num
0,1.0,1.0,1
1,1.0,1.0,1
2,1.0,1.0,1
3,1.0,1.0,1
4,0.0,0.0,0


In [27]:
tmp=list(col_name)
tmp.append('ind')
en.columns=tmp

In [28]:
en.head()

,model_1,model_2,ind
0,1.0,1.0,1
1,1.0,1.0,1
2,1.0,1.0,1
3,1.0,1.0,1
4,0.0,0.0,0


# Combining OR Metaclassifier as Logistic Regression

In [31]:
from sklearn.linear_model import LogisticRegression

In [32]:
clf=LogisticRegression(fit_intercept=False,solver='lbfgs')

In [34]:
clf.fit(en[['model_1','model_2']],en['ind'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
en_2=pd.DataFrame()

In [36]:
en_2['model_1'] = pd.DataFrame(model_1.predict_proba(X_test))[1]
en_2['model_2'] =  pd.DataFrame(model_2.predict_proba(X_test))[1]
col_name = en_2.columns
en_2['combined'] =clf.predict(en_2[['model_1', 'model_2']])

In [38]:
col_name = en_2.columns
tmp = list(col_name)
tmp.append('ind')

In [39]:
tmp

['model_1', 'model_2', 'combined', 'ind']

In [40]:
en_2 = pd.concat([en_2, pd.DataFrame(y_test).reset_index(drop=True)], axis=1)

In [42]:
en_2.columns = tmp

In [44]:
print(pd.crosstab(en_2['ind'], en_2['combined']))

combined  0    1
ind             
0         7   50
1         6  305


In [45]:
print(round(accuracy_score(en_2['ind'], en_2['combined']), 4))

0.8478


In [46]:
print(classification_report(en_2['ind'], en_2['combined']))

              precision    recall  f1-score   support

           0       0.54      0.12      0.20        57
           1       0.86      0.98      0.92       311

    accuracy                           0.85       368
   macro avg       0.70      0.55      0.56       368
weighted avg       0.81      0.85      0.81       368



# Single comb.

In [47]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

In [48]:
class_weight = {0:0.839, 1:0.161}

In [49]:
forest = RandomForestClassifier(class_weight=class_weight, n_estimators=100)

In [50]:
ada = AdaBoostClassifier(base_estimator=forest, n_estimators=100,
                         learning_rate=0.5, random_state=42)

In [51]:
ada.fit(X_train, y_train.ravel())

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=RandomForestClassifier(bootstrap=True,
                                                         ccp_alpha=0.0,
                                                         class_weight={0: 0.839,
                                                                       1: 0.161},
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features='auto',
                                                         max_leaf_nodes=None,
                                                         max_samples=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                              

In [52]:
print_score(ada, X_train, X_test, y_train, y_test, train=True)
print_score(ada, X_train, X_test, y_train, y_test, train=False)

Train Result:

accuracy score: 1.0000

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       180
           1       1.00      1.00      1.00       922

    accuracy                           1.00      1102
   macro avg       1.00      1.00      1.00      1102
weighted avg       1.00      1.00      1.00      1102


Confusion Matrix: 
 [[180   0]
 [  0 922]]

ROC AUC: 1.0000

Test Result:

accuracy score: 0.8614

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      0.11      0.19        57
           1       0.86      1.00      0.92       311

    accuracy                           0.86       368
   macro avg       0.93      0.55      0.56       368
weighted avg       0.88      0.86      0.81       368


Confusion Matrix: 
 [[  6  51]
 [  0 311]]

ROC AUC: 0.5526



In [53]:
bag_clf = BaggingClassifier(base_estimator=ada, n_estimators=50,
                            max_samples=1.0, max_features=1.0, bootstrap=True,
                            bootstrap_features=False, n_jobs=-1,
                            random_state=42)

In [54]:
bag_clf.fit(X_train, y_train.ravel())

BaggingClassifier(base_estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                    base_estimator=RandomForestClassifier(bootstrap=True,
                                                                                          ccp_alpha=0.0,
                                                                                          class_weight={0: 0.839,
                                                                                                        1: 0.161},
                                                                                          criterion='gini',
                                                                                          max_depth=None,
                                                                                          max_features='auto',
                                                                                          max_leaf_nodes=None,
                                                   

In [55]:
print_score(bag_clf, X_train, X_test, y_train, y_test, train=True)
print_score(bag_clf, X_train, X_test, y_train, y_test, train=False)

Train Result:

accuracy score: 0.9555

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      0.73      0.84       180
           1       0.95      1.00      0.97       922

    accuracy                           0.96      1102
   macro avg       0.97      0.86      0.91      1102
weighted avg       0.96      0.96      0.95      1102


Confusion Matrix: 
 [[131  49]
 [  0 922]]

ROC AUC: 0.8639

Test Result:

accuracy score: 0.8587

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      0.09      0.16        57
           1       0.86      1.00      0.92       311

    accuracy                           0.86       368
   macro avg       0.93      0.54      0.54       368
weighted avg       0.88      0.86      0.80       368


Confusion Matrix: 
 [[  5  52]
 [  0 311]]

ROC AUC: 0.5439

